In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import gzip
import gensim 
import logging
import re
import pandas as pd
import os
import gensim.downloader as api
import string

### Donot run this

In [0]:
os.getcwd()
os.chdir('/content/drive/My Drive/NLP Assign 2')

### Load the data into accessible form

- Here the questions from the analogy set have been loaded into a list.[1]
- The list is then separated depending upon the required categories.
- A dictionary and a list has been created for training(**final**),desired labels(**target**). This is to run the model and calculate accuracies on the entire data and individual category sets.

In [0]:
f = open('word-test.v1.txt', 'r', encoding="utf8")
x = f.readlines()
f.close()
x.remove(x[0])
new=[]
for ele in x:
    new.append(ele.strip().replace("\t", ""))
splitted = []
for item in new:
    if ':' in item:
        splitted.append([])
    splitted[-1].append(item)

In [0]:
word_list=['capital-world', 'currency', 'city-in-state', 'family', 'gram1-adjective-to-adverb', 'gram2-opposite', 'gram3-comparative', 'gram6-nationality-adjective']
word_list_dict={'capital-world':[], 'currency':[], 'city-in-state':[], 'family':[], 'gram1-adjective-to-adverb':[], 'gram2-opposite':[], 'gram3-comparative':[], 'gram6-nationality-adjective':[]}
final_list=[]
target_list=[]
for i in range(len(splitted)):
    key=splitted[i][0].replace(': ','')
    splitted[i].remove(splitted[i][0])
    if key in word_list:
#         print(key)
        for j in range(len(splitted[i])):
            phrase=splitted[i][j].split()
#             print(phrase)
            word_list_dict[key].append(phrase)
            final_list.append(phrase[:3])
            target_list.append(str(phrase[3:4][0]))

In [0]:
final_dict={'capital-world':[], 'currency':[], 'city-in-state':[], 'family':[], 'gram1-adjective-to-adverb':[], 'gram2-opposite':[], 'gram3-comparative':[], 'gram6-nationality-adjective':[]}
target_dict={'capital-world':[], 'currency':[], 'city-in-state':[], 'family':[], 'gram1-adjective-to-adverb':[], 'gram2-opposite':[], 'gram3-comparative':[], 'gram6-nationality-adjective':[]}
for key in word_list_dict:
    for i in range(len(word_list_dict[key])):
        final_dict[key].append(word_list_dict[key][i][:3])
        target_dict[key].append(str(word_list_dict[key][i][3:4][0]))
  

# Problem 7:

## Word2vec
- First choice is implementing Word2vec model using pretrained vectors from Google News Dataset.[2]
- The model has been imported using the Gensim Library.[3]
- The model has about 100 billion words, but due to memory constraints, the model has been trained on 50000 words.
- The cosine similarity between the vectors has been calculated using Gensim's most_similar() function.[3]

In [0]:
def test_model(embed_model):
  model=embed_model
  pred=[]
  for i in range(len(final_list)):
      if final_list[i][0] in model.vocab and final_list[i][2] in model.vocab and final_list[i][1] in model.vocab :
          pred.append(model.most_similar(positive=[final_list[i][0], final_list[i][2]], negative=[final_list[i][1]], topn=1)[0][0])
      else:
          pred.append('NO MATCHING PHRASE FOUND')
  pred_dict={'capital-world':[], 'currency':[], 'city-in-state':[], 'family':[], 'gram1-adjective-to-adverb':[], 'gram2-opposite':[], 'gram3-comparative':[], 'gram6-nationality-adjective':[]}
  for word in word_list:
    for i in range(len(final_dict[word])):
      if final_dict[word][i][0] in model.vocab and final_dict[word][i][2] in model.vocab and final_dict[word][i][1] in model.vocab :
          pred_dict[word].append(model.most_similar(positive=[final_dict[word][i][0], final_dict[word][i][2]], negative=[final_dict[word][i][1]], topn=1)[0][0])
      else:
          pred_dict[word].append('NO MATCHING PHRASE FOUND')
  return pred,pred_dict

In [0]:
def total_accuracy(pred):
    correct1=0
    translator=str.maketrans('','',string.punctuation)
    for i in range(len(target_list)):
      a=target_list[i]
      b=pred[i]
      if a==b or a in b:
          correct1+=1
    accuracy1=100*correct1/len(pred)
    accuracy2=100*correct1/(len(pred)-pred.count('NO MATCHING PHRASE FOUND'))
    return accuracy1, accuracy2


In [0]:
def sep_accuracy(pred_dict):
  accuracy1={'capital-world':[], 'currency':[], 'city-in-state':[], 'family':[], 'gram1-adjective-to-adverb':[], 'gram2-opposite':[], 'gram3-comparative':[], 'gram6-nationality-adjective':[]}
  accuracy2={'capital-world':[], 'currency':[], 'city-in-state':[], 'family':[], 'gram1-adjective-to-adverb':[], 'gram2-opposite':[], 'gram3-comparative':[], 'gram6-nationality-adjective':[]}
  for word in word_list:
    correct1=0
    not_found='NO MATCHING PHRASE FOUND'
    translator=str.maketrans('','',string.punctuation)
    for i in range(len(target_dict[word])):
      a=target_dict[word][i]
      b=pred_dict[word][i]
      if a==b or a in b:
    #       print(b)
          correct1+=1
    try:
      accuracy1[word]=100*correct1/len(pred_dict[word])
    except:
      accuracy1[word]=0
    try:
      accuracy2[word]=100*correct1/(len(pred_dict[word])-pred_dict[word].count(not_found))
    except:
      accuracy2[word]=0
    
  return accuracy1,accuracy2

In [11]:
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True, limit=50000)  

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


### Train model

In [12]:
pred,pred_dict=test_model(model)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


### Accuracies
- Accuracy1 describes accuracy when the model does not predict the word due to lack of vocabulary. Instead the model names this as **NO MATCHING PHRASE FOUND**
- Accuracy2 described the accuracy of the word embedding model to detect correct words after excluding the words which are not found in its vocabulary.
- Thus ideally Accuracy2>=Accuracy1
- The accuracies of separate categories are higher than when the entire set has been considered for training.

In [13]:
Accuracy1, Accuracy2=total_accuracy(pred)
print("COMBINED ACCURACY")
print()
print("Accuracy1",Accuracy1)
print()
print("Accuracy2",Accuracy2)

COMBINED ACCURACY

Accuracy1 1.427698885326004

Accuracy2 1.8790192926045015


In [14]:
Accuracy1, Accuracy2=sep_accuracy(pred_dict)
print("SEPARATE ACCURACIES")
print()
print("Accuracy1",Accuracy1)
print()
print("Accuracy2",Accuracy2)

SEPARATE ACCURACIES

Accuracy1 {'capital-world': 2.4314765694076037, 'currency': 0.0, 'city-in-state': 1.053911633563032, 'family': 5.138339920948616, 'gram1-adjective-to-adverb': 0.3024193548387097, 'gram2-opposite': 1.1083743842364533, 'gram3-comparative': 0.07507507507507508, 'gram6-nationality-adjective': 0.7504690431519699}

Accuracy2 {'capital-world': 5.132991133924405, 'currency': 0.0, 'city-in-state': 1.086048454469507, 'family': 5.895691609977324, 'gram1-adjective-to-adverb': 0.3024193548387097, 'gram2-opposite': 1.1904761904761905, 'gram3-comparative': 0.07507507507507508, 'gram6-nationality-adjective': 0.7889546351084813}


## GloVe

- Glove is a model representing distributed words. The glove twitter word vectors have been used for training.[4]

In [15]:
model1 = api.load("glove-twitter-25")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [18]:
pred,pred_dict=test_model(model1)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [16]:
Accuracy1, Accuracy2=total_accuracy(pred)
print("COMBINED ACCURACY")
print()
print("Accuracy1",Accuracy1)
print()
print("Accuracy2",Accuracy2)

COMBINED ACCURACY

Accuracy1 1.427698885326004

Accuracy2 1.8790192926045015


In [17]:
Accuracy1, Accuracy2=sep_accuracy(pred_dict)
print("SEPARATE ACCURACIES")
print()
print("Accuracy1",Accuracy1)
print()
print("Accuracy2",Accuracy2)

SEPARATE ACCURACIES

Accuracy1 {'capital-world': 2.4314765694076037, 'currency': 0.0, 'city-in-state': 1.053911633563032, 'family': 5.138339920948616, 'gram1-adjective-to-adverb': 0.3024193548387097, 'gram2-opposite': 1.1083743842364533, 'gram3-comparative': 0.07507507507507508, 'gram6-nationality-adjective': 0.7504690431519699}

Accuracy2 {'capital-world': 5.132991133924405, 'currency': 0.0, 'city-in-state': 1.086048454469507, 'family': 5.895691609977324, 'gram1-adjective-to-adverb': 0.3024193548387097, 'gram2-opposite': 1.1904761904761905, 'gram3-comparative': 0.07507507507507508, 'gram6-nationality-adjective': 0.7889546351084813}


# Problem 8:

- The model trained from words of Google News dataset has been used to find similar words of the following:
  - increase, above, maximum, before, agree
- It is observed that its antonym appears in the top ten list of similar words.
- This is because word embeddings donot take the context of the antonym in consideration. The models are trained on the word distributions over a large amount of data, the data which consists of antonyms having similar semantic context.
- For example the pair 'increase' and 'decrease' even though being antonyms, might be used in a similar semantic textual context

### Word2vec

In [18]:
model.most_similar('increase')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('decrease', 0.8370318412780762),
 ('increases', 0.7709376811981201),
 ('increased', 0.7578041553497314),
 ('reduction', 0.6908220648765564),
 ('increasing', 0.6871616244316101),
 ('decreases', 0.681617259979248),
 ('rise', 0.6352647542953491),
 ('decreasing', 0.621862530708313),
 ('decline', 0.6128641366958618),
 ('uptick', 0.5923734903335571)]

In [19]:
model.most_similar('above')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('below', 0.8064708709716797),
 ('Above', 0.5198284387588501),
 ('beyond', 0.4812318682670593),
 ('beneath', 0.45852547883987427),
 ('hover', 0.432657390832901),
 ('hovering', 0.42778879404067993),
 ('elevated', 0.42664289474487305),
 ('skyward', 0.4200187921524048),
 ('exceeds', 0.419230580329895),
 ('lower', 0.4161628782749176)]

In [20]:
model.most_similar('maximum')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('minimum', 0.6927940845489502),
 ('Maximum', 0.6223057508468628),
 ('max', 0.5242857933044434),
 ('optimum', 0.5091618299484253),
 ('limit', 0.49965229630470276),
 ('exceeds', 0.48546966910362244),
 ('Minimum', 0.4671606719493866),
 ('exceed', 0.45736148953437805),
 ('maximum_sentence', 0.45702826976776123),
 ('allowable', 0.4569830298423767)]

In [21]:
model.most_similar('before')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('after', 0.7583303451538086),
 ('Before', 0.6525088548660278),
 ('later', 0.6113136410713196),
 ('when', 0.5855079889297485),
 ('shortly', 0.582067608833313),
 ('prior', 0.5761102437973022),
 ('until', 0.566190242767334),
 ('then', 0.5548640489578247),
 ('After', 0.5400581359863281),
 ('last', 0.4965524971485138)]

In [22]:
model.most_similar('agree')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('disagree', 0.7711759209632874),
 ('concur', 0.7131549119949341),
 ('agrees', 0.5929451584815979),
 ('disagreed', 0.5711544156074524),
 ('Agree', 0.5635050535202026),
 ('disagreeing', 0.5525435209274292),
 ('agreed', 0.5445384383201599),
 ('insist', 0.5273953676223755),
 ('accept', 0.5188775062561035),
 ('sympathize', 0.5097719430923462)]

### GloVe

In [23]:
model1.most_similar('increase')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('increased', 0.9396798610687256),
 ('boost', 0.9143067598342896),
 ('growth', 0.9136716723442078),
 ('increasing', 0.9088935256004333),
 ('decrease', 0.9020586013793945),
 ('costs', 0.8986735343933105),
 ('decline', 0.8936024904251099),
 ('risk', 0.8911150097846985),
 ('debt', 0.8874548077583313),
 ('profit', 0.8826180100440979)]

In [24]:
model1.most_similar('above')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('within', 0.9225873947143555),
 ('through', 0.9183578491210938),
 ('ground', 0.9041840434074402),
 ('closer', 0.9011828899383545),
 ('inside', 0.8932865262031555),
 ('stands', 0.891708254814148),
 ('clear', 0.8907409310340881),
 ('falling', 0.8897219896316528),
 ('between', 0.8897184729576111),
 ('towards', 0.8846611380577087)]

In [25]:
model1.most_similar('maximum')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('impact', 0.8267736434936523),
 ('balance', 0.8258897066116333),
 ('continuous', 0.822946310043335),
 ('sentiment', 0.8205845952033997),
 ('significant', 0.8185640573501587),
 ('permanent', 0.808681309223175),
 ('obstacle', 0.8083568215370178),
 ('depth', 0.8060948848724365),
 ('minimum', 0.803694486618042),
 ('boost', 0.7995665073394775)]

In [26]:
model1.most_similar('before')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('until', 0.9726036190986633),
 ('after', 0.9658995270729065),
 ('goes', 0.9597133994102478),
 ('while', 0.9596750140190125),
 ('away', 0.95955890417099),
 ('waiting', 0.9583847522735596),
 ('out', 0.9493682384490967),
 ('going', 0.9485186338424683),
 ('again', 0.9485183954238892),
 ('coming', 0.946082592010498)]

In [27]:
model1.most_similar('agree')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('should', 0.9189361929893494),
 ('must', 0.9177311062812805),
 ('surely', 0.9163580536842346),
 ('think', 0.9151894450187683),
 ('would', 0.9146677851676941),
 ('how', 0.9101889133453369),
 ('if', 0.9089272022247314),
 ('well', 0.9083027243614197),
 ('obviously', 0.9044593572616577),
 ('agreed', 0.9032773971557617)]

# Problem 9:
- Analogy set 1: Animals and their sounds

In [0]:
set1_train=[['bats','screech','bees'],['crows','caw','ducks'],['owls','hoot','pigeons']]
set1_target=['buzz','quack','coo']

In [29]:
set1_pred=[]
for i in range(len(set1_train)):
    if set1_train[i][0] in model.vocab and set1_train[i][2] in model.vocab and set1_train[i][1] in model.vocab :
        set1_pred.append(model.most_similar(positive=[set1_train[i][0], set1_train[i][2]], negative=[set1_train[i][1]], topn=1)[0][0])
    else:
        set1_pred.append('NO MATCHING PHRASE FOUND')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [30]:
correct_set1=0
translator=str.maketrans('','',string.punctuation)
for i in range(len(set1_target)):
  a=set1_target[i]
  b=set1_pred[i]
  if a==b or a in b:
      correct_set1+=1
accuracy1=100*correct_set1/len(set1_pred)
accuracy2=100*correct_set1/(len(set1_pred)-set1_pred.count('NO MATCHING PHRASE FOUND'))
print("Accuracy1",accuracy1)
print("Accuracy2",accuracy2)

Accuracy1 0.0
Accuracy2 0.0


- Analogy set 2: Animals and their babies

In [0]:
set2_train=[['bear','cub','bee'],['cat','kitten','dog'],['chicken','chick','pig']]
set2_target=['larva','puppy','piglet']

In [32]:
set2_pred=[]
for i in range(len(set2_train)):
    if set2_train[i][0] in model.vocab and set2_train[i][2] in model.vocab and set2_train[i][1] in model.vocab :
        set2_pred.append(model.most_similar(positive=[set2_train[i][0], set2_train[i][2]], negative=[set2_train[i][1]], topn=1)[0][0])
    else:
        set2_pred.append('NO MATCHING PHRASE FOUND')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [33]:
correct_set2=0
translator=str.maketrans('','',string.punctuation)
for i in range(len(set2_target)):
  a=set2_target[i]
  b=set2_pred[i]
  if a==b or a in b:
      correct_set2+=1
accuracy1=100*correct_set1/len(set2_pred)
accuracy2=100*correct_set1/(len(set2_pred)-set2_pred.count('NO MATCHING PHRASE FOUND'))
print("Accuracy1",accuracy1)
print("Accuracy2",accuracy2)

Accuracy1 0.0
Accuracy2 0.0


### The accuracies in both the case is 0 which means that the model has to be trained on a large data. Depending upon the analogies to be found, special text processing can also be carried out.

### References

[1] http://www.fit.vutbr.cz/~imikolov/rnnlm/word-test.v1.txt

[2]  Word2vec: https://code.google.com/archive/p/word2vec/

[3] https://radimrehurek.com/gensim/models/word2vec.html

[4] GloVe: http://nlp.stanford.edu/projects/glove/